# Modelo para predição de tempo de julgamento de processos judicais

O modelo será teinado com base nos dados disponíveis no Datajud, que é a base nacional de metadados processuais do poder judiciário brasileiro.

[Documentação do Datajud](https://datajud-wiki.cnj.jus.br/)

Os dados disponíveis são:

Tipo do Tribunal (Estadual, Federal, Trabalho, Eleitoal)
Estado
Grau do processo (G1, G2, TR, JE)
Unidade Judicial / Vara
Classe Processual
Assunto
Data Ajuizamento(Data de distribuição do processo)
Lista de movimentações

As classes, assuntos e movimentações eguem uma tabela nacional, com seus códigos e classificação previamente estabelecida.
Essa padronização é a TPU - [Tabelas Processuais Unificadas](https://www.cnj.jus.br/sgt/consulta_publica_classes.php)


## SQL extração classes em seus níveis

WITH nivel_1 AS (
	SELECT clcodigo AS codnivel1 FROM bpu.classe WHERE clcodpai IS NULL AND clcodigo < 1000000
),nivel_2 AS (
	SELECT codnivel1, clcodigo AS codnivel2 FROM nivel_1 LEFT JOIN bpu.classe ON clcodpai = codnivel1
	UNION
	SELECT codnivel1, NULL AS codnivel2 FROM nivel_1
),
nivel_3 AS (
	SELECT codnivel1, codnivel2, clcodigo AS codnivel3 FROM nivel_2 LEFT JOIN bpu.classe ON clcodpai = codnivel2
	UNION
	SELECT codnivel1, codnivel2, NULL AS codnivel3 FROM nivel_2
),
nivel_4 AS (
	SELECT codnivel1, codnivel2, codnivel3, clcodigo AS codnivel4 FROM nivel_3 LEFT JOIN bpu.classe ON clcodpai = codnivel3
	UNION
	SELECT codnivel1, codnivel2, codnivel3, NULL AS codnivel4 FROM nivel_3
),
nivel_5 AS (
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, clcodigo AS codnivel5 FROM nivel_4 LEFT JOIN bpu.classe ON clcodpai = codnivel4
	UNION
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, NULL AS codnivel5 FROM nivel_4
),
nivel_6 AS (
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, codnivel5, clcodigo AS codnivel6 FROM nivel_5 LEFT JOIN bpu.classe ON clcodpai = codnivel5
	UNION
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, codnivel5, NULL AS codnivel6 FROM nivel_5
)
SELECT
	codnivel1,
	codnivel2,
	codnivel3,
	codnivel4,
	codnivel5,
	COALESCE(codnivel6,codnivel5,codnivel4,codnivel3,codnivel2,codnivel1) AS codclasse,
	cldescricao AS descclasse
FROM nivel_6
	LEFT JOIN bpu.classe ON COALESCE(codnivel6,codnivel5,codnivel4,codnivel3,codnivel2,codnivel1)=clcodigo
ORDER BY codnivel1,codnivel2,codnivel3,codnivel4,codnivel5,codclasse;

In [3]:
import pandas as pd


In [4]:
classes_tpu = pd.read_csv('https://drive.google.com/uc?export=download&id=19Y7Zjwo8bZFHZ6rwO_fMsVljU0frLAon')
assuntos_tpu = pd.read_csv('https://drive.google.com/uc?export=download&id=17FtEye4Vq75cHUjvhiC4Rl5oKlCPpudK', sep=',')

In [5]:
classes_tpu[classes_tpu['codclasse']==7]

,codnivel1,codnivel2,codnivel3,codnivel4,codnivel5,codclasse,descclasse
163,2,1106.0,1107.0,7.0,NaN,7,PROCEDIMENTO COMUM CÍVEL


In [6]:
mov_julgamentos = [193,196,198,200,202,208,210,212,214,218,219,220,221,228,230,235,236,237,238,239,240,241,242,244,385,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,471,472,473,853,871,884,900,901,972,973,1042,1043,1044,1045,1046,1047,1048,1049,1050,10953,10961,10964,10965,11373,11374,11375,11376,11377,11378,11379,11380,11381,11394,11396,11401,11402,11403,11404,11405,11406,11407,11408,11409,11411,11795,11796,11801,11876,11877,11878,11879,12028,12032,12033,12034,12041,12184,12187,12252,12253,12254,12256,12257,12258,12298,12319,12321,12322,12323,12324,12325,12326,12327,12328,12329,12330,12331,12433,12434,12435,12436,12437,12438,12439,12440,12441,12442,12443,12450,12451,12452,12453,12458,12459,12475,12615,12616,12617,12649,12650,12651,12652,12653,12654,12660,12661,12662,12663,12664,12665,12666,12667,12668,12669,12670,12671,12672,12673,12674,12675,12676,12677,12678,12679,12680,12681,12682,12683,12684,12685,12686,12687,12688,12689,12690,12691,12692,12693,12694,12695,12696,12697,12698,12699,12700,12701,12702,12703,12704,12705,12706,12707,12708,12709,12710,12711,12712,12713,12714,12715,12716,12717,12718,12719,12720,12721,12722,12723,12724,12735,12738,12792,14099,14210,14211,14213,14214,14215,14216,14217,14218,14219,14680,14777,14778,14848,14937,15022,15023,15024,15026,15027,15028,15029,15030,15165,15166,15211,15212,15213,15214,15245,15249,15250,15251,15252,15253,15254,15255,15256,15257,15258,15259,15260,15261,15262,15263,15264,15265,15266,15322,15408,1002013,1003301,1003302,1050009,1050080,1050083,1050147,4050028,4050079,4050080,4050083]

In [7]:
assuntos_tpu[assuntos_tpu['codassunto']==5787]

,codnivel1,codnivel2,codnivel3,codnivel4,codnivel5,codassunto,descassunto
2049,899,5626.0,5779.0,5787.0,NaN,5787,EXONERAÇÃO


In [9]:
import requests
import json

url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrr/_search"

payload = json.dumps({
    "query": {
        "bool": {
            "must": [
                {"match": {"classe.codigo": 7}}
            ]
        }
    }
}
)

#Substituir <API Key> pela Chave Pública
headers = {
  'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"took":108,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":1.0,"hits":[{"_index":"api_publica_tjrr","_type":"_doc","_id":"TJRR_G1_08096157620168230010","_score":1.0,"_source":{"classe":{"codigo":7,"nome":"Procedimento Comum Cível"},"numeroProcesso":"08096157620168230010","sistema":{"codigo":-1,"nome":"Inválido"},"formato":{"codigo":1,"nome":"Eletrônico"},"tribunal":"TJRR","dataHoraUltimaAtualizacao":"2024-08-27T02:11:17.635Z","grau":"G1","@timestamp":"2024-08-27T03:07:45.856Z","dataAjuizamento":"2016-04-07T08:45:47.000Z","movimentos":[{"complementosTabelados":[{"codigo":19,"valor":58,"nome":"Petição inicial","descricao":"tipo_de_peticao"}],"codigo":85,"nome":"Petição","dataHora":"2016-04-07T08:45:47.000Z"},{"complementosTabelados":[{"codigo":18,"valor":40,"nome":"outros motivos","descricao":"motivo_da_remessa"}],"codigo":123,"nome":"Remessa","dataHora":"2016-04-07T08:45:47.000Z"},{"complementosT

In [10]:
import json
resposta  = json.loads(response.text)

In [11]:
lista = resposta['hits']['hits']

In [14]:
df = pd.DataFrame.from_records(lista)
df.head()

,_index,_type,_id,_score,_source
0,api_publica_tjrr,_doc,TJRR_G1_08096157620168230010,1.0,"{'classe': {'codigo': 7, 'nome': 'Procedimento..."
1,api_publica_tjrr,_doc,TJRR_G1_08002117220158230030,1.0,"{'classe': {'codigo': 7, 'nome': 'Procedimento..."
2,api_publica_tjrr,_doc,TJRR_G1_08165350320158230010,1.0,"{'classe': {'codigo': 7, 'nome': 'Procedimento..."
3,api_publica_tjrr,_doc,TJRR_G1_08099099420178230010,1.0,"{'classe': {'codigo': 7, 'nome': 'Procedimento..."
4,api_publica_tjrr,_doc,TJRR_G1_08098709220208230010,1.0,"{'classe': {'codigo': 7, 'nome': 'Procedimento..."


In [15]:
df = df['_source'].apply(pd.Series)

In [16]:
df.head()

,classe,numeroProcesso,sistema,formato,tribunal,dataHoraUltimaAtualizacao,grau,@timestamp,dataAjuizamento,movimentos,id,nivelSigilo,orgaoJulgador,assuntos
0,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08096157620168230010,"{'codigo': -1, 'nome': 'Inválido'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,2024-08-27T02:11:17.635Z,G1,2024-08-27T03:07:45.856Z,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",TJRR_G1_08096157620168230010,0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí..."
1,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08002117220158230030,"{'codigo': -1, 'nome': 'Inválido'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,2024-08-27T02:11:17.605Z,G1,2024-08-27T03:07:45.860Z,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",TJRR_G1_08002117220158230030,0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R..."
2,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08165350320158230010,"{'codigo': -1, 'nome': 'Inválido'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,2024-08-27T02:11:17.903Z,G1,2024-08-27T03:07:40.532Z,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",TJRR_G1_08165350320158230010,0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da..."
3,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08099099420178230010,"{'codigo': -1, 'nome': 'Inválido'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,2024-08-27T02:10:54.877Z,G1,2024-08-27T03:08:38.056Z,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",TJRR_G1_08099099420178230010,0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan..."
4,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08098709220208230010,"{'codigo': -1, 'nome': 'Inválido'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,2024-08-27T02:12:37.711Z,G1,2024-08-27T03:02:39.335Z,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",TJRR_G1_08098709220208230010,0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej..."


In [17]:
df.drop(columns=['@timestamp','id','sistema','dataHoraUltimaAtualizacao'],inplace=True)

In [18]:
df[['codigo_classe','nome_classe']] = df['classe'].apply(pd.Series)

In [19]:
df

,classe,numeroProcesso,formato,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe,nome_classe
0,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08096157620168230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,Procedimento Comum Cível
1,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08002117220158230030,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,Procedimento Comum Cível
2,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08165350320158230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7,Procedimento Comum Cível
3,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08099099420178230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,Procedimento Comum Cível
4,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08098709220208230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,Procedimento Comum Cível
5,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08029520420228230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,Procedimento Comum Cível
6,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08006242620208230090,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,Procedimento Comum Cível
7,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08313996520238230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,Procedimento Comum Cível
8,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08000806020248230005,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,Procedimento Comum Cível
9,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}",08134336020218230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,Procedimento Comum Cível


In [20]:
df.drop(columns=['classe','nome_classe'],inplace=True)

In [21]:
df

,numeroProcesso,formato,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe
0,08096157620168230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7
1,08002117220158230030,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7
2,08165350320158230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7
3,08099099420178230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7
4,08098709220208230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7
5,08029520420228230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7
6,08006242620208230090,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7
7,08313996520238230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7
8,08000806020248230005,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7
9,08134336020218230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7


In [22]:
df[['codigo_formato','nome_formato']] = df['formato'].apply(pd.Series)

In [23]:
df

,numeroProcesso,formato,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe,codigo_formato,nome_formato
0,08096157620168230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1,Eletrônico
1,08002117220158230030,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1,Eletrônico
2,08165350320158230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1,Eletrônico
3,08099099420178230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1,Eletrônico
4,08098709220208230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1,Eletrônico
5,08029520420228230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1,Eletrônico
6,08006242620208230090,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,Eletrônico
7,08313996520238230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1,Eletrônico
8,08000806020248230005,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1,Eletrônico
9,08134336020218230010,"{'codigo': 1, 'nome': 'Eletrônico'}",TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,Eletrônico


In [24]:
df.drop(columns=['formato','nome_formato'],inplace=True)

In [25]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe,codigo_formato
0,08096157620168230010,TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1
1,08002117220158230030,TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1
2,08165350320158230010,TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1
3,08099099420178230010,TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1
4,08098709220208230010,TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1
5,08029520420228230010,TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1
6,08006242620208230090,TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1
7,08313996520238230010,TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1
8,08000806020248230005,TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1
9,08134336020218230010,TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1


In [26]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe,codigo_formato
0,08096157620168230010,TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1
1,08002117220158230030,TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1
2,08165350320158230010,TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1
3,08099099420178230010,TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1
4,08098709220208230010,TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1
5,08029520420228230010,TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1
6,08006242620208230090,TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1
7,08313996520238230010,TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1
8,08000806020248230005,TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1
9,08134336020218230010,TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1


In [27]:
# Explode coluna orgaoJulgador
df[['orgao_codigoMunicipioIBGE','orgao_codigo','orgao_nome']] = df['orgaoJulgador'].apply(pd.Series)

In [28]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,orgaoJulgador,assuntos,codigo_classe,codigo_formato,orgao_codigoMunicipioIBGE,orgao_codigo,orgao_nome
0,08096157620168230010,TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1,1400100,2764,1ª VARA DA FAZENDA PÚBLICA
1,08002117220158230030,TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400308, 'codigo': 278...","[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1,1400308,2782,VARA CÍVEL
2,08165350320158230010,TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1,1400100,2768,4ª VARA CÍVEL
3,08099099420178230010,TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1,1400100,83366,2º NÚCLEO DE JUSTIÇA 4.0
4,08098709220208230010,TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1,1400100,2764,1ª VARA DA FAZENDA PÚBLICA
5,08029520420228230010,TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 277...","[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1,1400100,2770,2ª VARA DA FAZENDA PÚBLICA
6,08006242620208230090,TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400159, 'codigo': 129...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,1400159,12910,VARA CÍVEL
7,08313996520238230010,TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 276...","[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1,1400100,2765,1ª VARA CÍVEL
8,08000806020248230005,TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400050, 'codigo': 278...","[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1,1400050,2780,VARA CÍVEL
9,08134336020218230010,TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"{'codigoMunicipioIBGE': 1400100, 'codigo': 833...","[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,1400100,83366,2º NÚCLEO DE JUSTIÇA 4.0


In [29]:
df.drop(columns=['orgaoJulgador','orgao_codigoMunicipioIBGE','orgao_nome'],inplace=True)

In [30]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,assuntos,codigo_classe,codigo_formato,orgao_codigo
0,08096157620168230010,TJRR,G1,2016-04-07T08:45:47.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1,2764
1,08002117220158230030,TJRR,G1,2015-02-18T22:51:07.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1,2782
2,08165350320158230010,TJRR,G1,2015-06-22T09:57:32.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1,2768
3,08099099420178230010,TJRR,G1,2017-04-17T18:42:02.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1,83366
4,08098709220208230010,TJRR,G1,2020-04-06T12:39:48.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1,2764
5,08029520420228230010,TJRR,G1,2022-02-02T17:15:14.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1,2770
6,08006242620208230090,TJRR,G1,2020-10-30T18:06:52.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,12910
7,08313996520238230010,TJRR,G1,2023-08-29T18:58:57.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1,2765
8,08000806020248230005,TJRR,G1,2024-01-26T18:21:15.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1,2780
9,08134336020218230010,TJRR,G1,2021-05-25T17:51:26.000Z,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,83366


In [31]:
# Converter coluna dataAjuizamento em data
df['dataAjuizamento'] = pd.to_datetime(df['dataAjuizamento']).dt.strftime('%Y-%m-%d')


In [32]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,assuntos,codigo_classe,codigo_formato,orgao_codigo
0,08096157620168230010,TJRR,G1,2016-04-07,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1,2764
1,08002117220158230030,TJRR,G1,2015-02-18,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1,2782
2,08165350320158230010,TJRR,G1,2015-06-22,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1,2768
3,08099099420178230010,TJRR,G1,2017-04-17,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1,83366
4,08098709220208230010,TJRR,G1,2020-04-06,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1,2764
5,08029520420228230010,TJRR,G1,2022-02-02,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1,2770
6,08006242620208230090,TJRR,G1,2020-10-30,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,12910
7,08313996520238230010,TJRR,G1,2023-08-29,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1,2765
8,08000806020248230005,TJRR,G1,2024-01-26,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1,2780
9,08134336020218230010,TJRR,G1,2021-05-25,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,83366


In [36]:
movimentos =  [
              {
                  "complementosTabelados": [
                      {
                          "codigo": 2,
                          "valor": 1,
                          "nome": "competência exclusiva",
                          "descricao": "tipo_de_distribuicao_redistribuicao"
                      }
                  ],
                  "codigo": 193,
                  "nome": "Distribuição",
                  "dataHora": "2018-10-30T14:06:24.000Z"
              },

              {
                  "codigo": 14732,
                  "nome": "Conversão de Autos Físicos em Eletrônicos",
                  "dataHora": "2020-08-05T01:15:18.000Z"
              }
          ]

movimentos

[{'complementosTabelados': [{'codigo': 2,
    'valor': 1,
    'nome': 'competência exclusiva',
    'descricao': 'tipo_de_distribuicao_redistribuicao'}],
  'codigo': 193,
  'nome': 'Distribuição',
  'dataHora': '2018-10-30T14:06:24.000Z'},
 {'codigo': 14732,
  'nome': 'Conversão de Autos Físicos em Eletrônicos',
  'dataHora': '2020-08-05T01:15:18.000Z'}]

In [37]:
assuntos =[{'codigo': 10439, 'nome': 'Indenização por Dano Material'}, {'codigo': 10433, 'nome': 'Indenização por Dano Moral'}]

In [38]:
def extrair_assunto(assuntos):
  return assuntos[0]['codigo']

In [39]:
df['codigo_assunto'] = df['assuntos'].apply(extrair_assunto)

In [40]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,assuntos,codigo_classe,codigo_formato,orgao_codigo,codigo_assunto
0,08096157620168230010,TJRR,G1,2016-04-07,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10013, 'nome': 'Enriquecimento ilí...",7,1,2764,10013
1,08002117220158230030,TJRR,G1,2015-02-18,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 4964, 'nome': 'Cédula de Crédito R...",7,1,2782,4964
2,08165350320158230010,TJRR,G1,2015-06-22,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10433, 'nome': 'Indenização por Da...",7,1,2768,10433
3,08099099420178230010,TJRR,G1,2017-04-17,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9992, 'nome': 'Indenização por Dan...",7,1,83366,9992
4,08098709220208230010,TJRR,G1,2020-04-06,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9524, 'nome': 'Indenização do Prej...",7,1,2764,9524
5,08029520420228230010,TJRR,G1,2022-02-02,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10255, 'nome': 'Invalidez Permanen...",7,1,2770,10255
6,08006242620208230090,TJRR,G1,2020-10-30,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,12910,10439
7,08313996520238230010,TJRR,G1,2023-08-29,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 9596, 'nome': 'Prestação de Serviç...",7,1,2765,9596
8,08000806020248230005,TJRR,G1,2024-01-26,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 7780, 'nome': 'Indenização por Dan...",7,1,2780,7780
9,08134336020218230010,TJRR,G1,2021-05-25,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,"[{'codigo': 10439, 'nome': 'Indenização por Da...",7,1,83366,10439


In [41]:
df.drop(columns=['assuntos'],inplace=True)

In [42]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,codigo_classe,codigo_formato,orgao_codigo,codigo_assunto
0,08096157620168230010,TJRR,G1,2016-04-07,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2764,10013
1,08002117220158230030,TJRR,G1,2015-02-18,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2782,4964
2,08165350320158230010,TJRR,G1,2015-06-22,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2768,10433
3,08099099420178230010,TJRR,G1,2017-04-17,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,83366,9992
4,08098709220208230010,TJRR,G1,2020-04-06,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2764,9524
5,08029520420228230010,TJRR,G1,2022-02-02,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2770,10255
6,08006242620208230090,TJRR,G1,2020-10-30,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,12910,10439
7,08313996520238230010,TJRR,G1,2023-08-29,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2765,9596
8,08000806020248230005,TJRR,G1,2024-01-26,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2780,7780
9,08134336020218230010,TJRR,G1,2021-05-25,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,83366,10439


In [43]:
def extrair_sentenca(movimentos):
  for movimento in movimentos:
    if movimento['codigo'] in mov_julgamentos:
      return movimento['dataHora']
  return None

In [44]:
extrair_sentenca(movimentos)

'2018-10-30T14:06:24.000Z'

In [49]:
df['data_julgamento'] = df['movimentos'].apply(extrair_sentenca)
df['dataJulgamento'] = pd.to_datetime(df['data_julgamento']).dt.strftime('%Y-%m-%d');
df.drop(columns=['data_julgamento'],inplace=True)
df.dropna(subset=['dataJulgamento'], inplace=True)

In [50]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,codigo_classe,codigo_formato,orgao_codigo,codigo_assunto,dataJulgamento
0,08096157620168230010,TJRR,G1,2016-04-07,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2764,10013,2019-08-26
1,08002117220158230030,TJRR,G1,2015-02-18,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2782,4964,2015-11-16
4,08098709220208230010,TJRR,G1,2020-04-06,"[{'complementosTabelados': [{'codigo': 19, 'va...",0,7,1,2764,9524,2024-08-14
